<a href="https://colab.research.google.com/github/Developer-Ujjwal/Generative-AI/blob/main/Rag_implementation_with_chromaDB_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain_community

In [ ]:
from langchain_community.document_loaders import UnstructuredURLLoader

In [ ]:
!pip install unstructured

In [ ]:
urls = ['https://www.secretshare.tech']
loader = UnstructuredURLLoader(urls=urls)
data = loader.load()

In [ ]:
data

[Document(metadata={'source': 'https://www.secretshare.tech'}, page_content="SecretShare Logo\n\nShare Secrets Securely\n\nEnd-to-end encrypted, one-time secret sharing with self-destruct capabilities. Your data is encrypted client-side and never stored unencrypted on our servers.\n\nAES-256 Encryption\n\nSelf-Destruct Timer\n\nOne-Time View\n\nInstant Sharing\n\nCreate a Secret\n\nYour message or file will be encrypted client-side and can only be viewed once.\n\nKey Features\n\nEnd-to-End Encryption\n\nYour data is encrypted client-side with AES-256 before leaving your device\n\nSelf-Destruct Timer\n\nSecrets automatically destroy after a set viewing duration\n\nOne-Time Viewing\n\nEach secret can only be viewed once, then it's gone forever\n\nUniversal File Support\n\nShare any file type - documents, images, videos, archives\n\nEnd-to-End Encryption\n\nMilitary-grade encryption ensures your secrets remain private\n\nUniversal File Support\n\nDocuments\n\nPDF, DOC, TXT, RTF\n\nImages\

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter= RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts = text_splitter.split_documents(data)

In [ ]:
texts

[Document(metadata={'source': 'https://www.secretshare.tech'}, page_content="SecretShare Logo\n\nShare Secrets Securely\n\nEnd-to-end encrypted, one-time secret sharing with self-destruct capabilities. Your data is encrypted client-side and never stored unencrypted on our servers.\n\nAES-256 Encryption\n\nSelf-Destruct Timer\n\nOne-Time View\n\nInstant Sharing\n\nCreate a Secret\n\nYour message or file will be encrypted client-side and can only be viewed once.\n\nKey Features\n\nEnd-to-End Encryption\n\nYour data is encrypted client-side with AES-256 before leaving your device\n\nSelf-Destruct Timer\n\nSecrets automatically destroy after a set viewing duration\n\nOne-Time Viewing\n\nEach secret can only be viewed once, then it's gone forever\n\nUniversal File Support\n\nShare any file type - documents, images, videos, archives\n\nEnd-to-End Encryption\n\nMilitary-grade encryption ensures your secrets remain private\n\nUniversal File Support\n\nDocuments\n\nPDF, DOC, TXT, RTF\n\nImages\

In [ ]:
!pip install transformers

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
model_id = 'google/flan-t5-large'

In [ ]:
embeddings=HuggingFaceEmbeddings(model_name=model_id)

In [ ]:
!pip -q install chromadb

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
persist_directory = 'db'
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embeddings,
                                 persist_directory=persist_directory)

In [ ]:
vectordb.persist()
vectordb = None

In [ ]:
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embeddings)

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
docs = retriever.get_relevant_documents("What is the use of encryption")

In [ ]:
retriever.search_type

'similarity'

In [ ]:
docs

[Document(metadata={'source': 'https://www.secretshare.tech'}, page_content='Archives\n\nZIP, RAR, 7Z\n\nClient-Side Encryption\n\nAll encryption happens in your browser. We never see your unencrypted data.\n\nZero Knowledge\n\nOur servers never have access to your encryption keys or unencrypted content.\n\nOpen Source\n\nTransparent, auditable code ensures trust and security for all users.'),
 Document(metadata={'source': 'https://www.secretshare.tech'}, page_content='Archives\n\nZIP, RAR, 7Z\n\nClient-Side Encryption\n\nAll encryption happens in your browser. We never see your unencrypted data.\n\nZero Knowledge\n\nOur servers never have access to your encryption keys or unencrypted content.\n\nOpen Source\n\nTransparent, auditable code ensures trust and security for all users.'),
 Document(metadata={'source': 'https://www.secretshare.tech'}, page_content="SecretShare Logo\n\nShare Secrets Securely\n\nEnd-to-end encrypted, one-time secret sharing with self-destruct capabilities. Your

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

generator = pipeline("text2text-generation", model=model_id, max_length=64)
llm = HuggingFacePipeline(pipeline=generator)


Device set to use cuda:0


In [ ]:
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
llm_response = qa_chain("What is the use of encryption")

Token indices sequence length is longer than the specified maximum sequence length for this model (681 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
llm_response

{'query': 'What is the use of encryption',
 'result': 'End-to-end encrypted, one-time secret sharing with self-destruct capabilities. Your data is encrypted client-side and never stored unencrypted on our servers',
 'source_documents': [Document(metadata={'source': 'https://www.secretshare.tech'}, page_content='Archives\n\nZIP, RAR, 7Z\n\nClient-Side Encryption\n\nAll encryption happens in your browser. We never see your unencrypted data.\n\nZero Knowledge\n\nOur servers never have access to your encryption keys or unencrypted content.\n\nOpen Source\n\nTransparent, auditable code ensures trust and security for all users.'),
  Document(metadata={'source': 'https://www.secretshare.tech'}, page_content='Archives\n\nZIP, RAR, 7Z\n\nClient-Side Encryption\n\nAll encryption happens in your browser. We never see your unencrypted data.\n\nZero Knowledge\n\nOur servers never have access to your encryption keys or unencrypted content.\n\nOpen Source\n\nTransparent, auditable code ensures trust 